In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough


chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=200,
    memory_key="memory",
    return_messages=True,
)


examples = [
    {"movie": "Top Gun", "answer": "🛩️👨‍✈️🔥"},
    {"movie": "The Dark Knight", "answer": "🦇👻🔫"},
    {"movie": "The Matrix", "answer": "🔮👾💻"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Summarize the movie '{movie}' in 3 emojis."),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that can answer questions about movies.",
        ),
        example_prompt,
        MessagesPlaceholder(variable_name="memory"),
        ("human", "Summarize the movie '{movie}' in 3 emojis."),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["memory"]


chain = RunnablePassthrough.assign(memory=load_memory) | final_prompt | chat


def invoke_chain(question):
    result = chain.invoke({"movie": question})
    memory.save_context({"input": question}, {"output": result.content})


invoke_chain("Inception")

🧠🌌🕰️🧠🌌🕰️


In [14]:
invoke_chain("The Dark Knight")

🦇🤡🃏🦇🤡🃏
